# Data Cleaning & Preprocessing Techniques

This project is a simple walk-through of various data cleaning and preprocessing techniques to consider before doing data analysis.

### Why

Raw data often contains errors, missing values, inconsistencies, and other issues that can negatively impact the quality and reliability of any downstream analyses. By cleaning up, and preprocessing data, one can ensure that the data they are working with is accurate, complete, and in a format that is appropriate for the intended analysis or modeling task. This can lead to more reliable and accurate results, as well as more efficient use of time and resources.

> **Note**: Data cleaning and preprocessing is an iterative process as new issues can be uncovered during analysis or modeling.

### What this IS NOT!

Data cleaning and preprocessing is not a way for you to introduce your own bias. This can happen unintentionally, but the techniques used should be conducted carefully to avoid overemphasizing or underemphasizing aspects of your data.

For example, if a dataset contains missing values, one common approach to address this is to impute the missing values. However, if the missing values are not random, but instead systematically related to other variables in the dataset, then the imputed values may introduce bias into the dataset.

### Data Cleaning (DC)
**Data Cleaning** involves Identifying and correcting errors and inconsistencies in the data such as:

- incorrect formatting
- missing values
- outliers
- duplicates

The goal of data cleaning is to ensure that the data is **accurate**, **complete**, and **consistent**.

### Data Preprocessing (DP)
Data preprocessing on the otherhand, involves transforming the data into a format that is more suitable for analysis or modeling. Data preprocessing techniques may include:

- scaling or standardizing variables
- encoding categorical variables 
- reducing the dimensionality of the data
- transforming the data into a different representation